# Association Mining
--------------------

### Aufgabe 1: Filmempfehlungen (20 Punkte)

Im Ordner "data" finden Sie einen Datensatz, der für mehr als 9000 Zuschauer eine Liste mit TV-Sendungen enthält, die sie angeschaut haben. Nutzen Sie diesen Datensatz, um ein Empfehlungssystem zu entwickeln, das gegeben eine TV-Sendung, die ein Nutzer angeschaut hat, weitere Sendungen vorschlägt, die dem Nutzer ebenfalls gefallen könnten. Basieren Sie Ihre Empfehlung dabei auf dem gegebenen Datensatz (d.h. die Empfehlung sollte im Stil "Zuschauer der Sendung XY schauten auch..." sein).

Nutzen Sie Ihr Empfehlungssystem um eine Empfehlung für Nutzer zu generieren, die die Sendung "Mr. Robot" geschaut haben.

Hinweise: 
* Verwenden Sie als ML-Verfahren Association Mining, um zu den Empfehlungen zu kommen.
* Überlegen Sie sich zunächst, welche Teilschritte vorgenommen werden müssen und notieren Sie diese schriftlich. 
* Verwenden Sie für die technische Umsetzung die Library [``mlxtend``: ](http://rasbt.github.io/mlxtend/)
* ``mlxtend`` erwartet eine Liste, die keine nan-Werte enthält. Um Ihnen einige Vorverarbeitungsschritte zu ersparen, finden Sie unten einige Zellen mit Code, die die Daten einlesen und ins richtige Format bringen.
* Überlegen Sie sich sinnvolle Parameter-, bzw Schwellwerte für die Anwendung von Association Mining in diesem Kontext und begründen Sie Ihre Wahl.
* Es ist nicht erforderlich eine GUI oder ähnliches zu entwickeln, sondern es reicht, wenn in Ihrem Notebook eine Eingabemöglichkeit für einen Filmtitel gegeben ist. 




####  Teilaufgabe a) Erforderliche Teilschritte (4 Punkte)

- Kandidaten-Generierung (Apriori-Algorithmus) (mit Hash-Tree)
- starke Regeln generieren -> pruning/Aprior 
- Ausgeben lassen, welche Produkte ebenfalls oft geschaut wurden
<br>
<br>
-> Hyperparameter: minSupport (50), minKonfidenz(50), Messungsart der Interessantheit(normale Konfidenz) -> erstmal mit standartwerten -> empirisch testen
<br>
-> Evtl. auf FP-Tree anstatt Kandidaten generierung umsteigen<br>
<br>
**Weitere benötigte Elemente:**<br>
- User-Eingabe
- Laden der Daten + Transformieren/in richtige Form bringen
- Produkt überprüfung -> existiert Produkt (?)
- Ähnlichkeitsausgabe / Ausgabe des Ergebnisses

#### Teilaufgabe b) Ihre Implementierung (13 Punkte)

In [1]:
import pandas as pd
import numpy as np

# Usage of the mlxtend module
from mlxtend.frequent_patterns import apriori, association_rules, fpgrowth
from mlxtend.preprocessing import TransactionEncoder

Ähnliche Produkte finden (welche ander Personen auch angesehen habe):

In [2]:
# Get proposals for film/series X
#     -> Persons who watched X watched Y too
def get_proposals(name:str) -> str:
    # Input Film/Serie to lowercase -> Film/Series titles going to be all lowercase -> for that a demand will be more uncomplicated
    name = name.lower()

    # Load data
    movies = pd.read_csv("./data/TV Shows - Association Rule Learning.csv", header = None)
    movies.head()

    # Remove nan values from list
    movies2 = []
    for label, row in movies.iterrows(): 
        line = [item for item in row if not(pd.isna(item))] 
        movies2.append(line)

    # One Hot Encoding -> Every Film/Series going to be a Feature with True/False
    encoder = TransactionEncoder().fit(movies2)
    prepared_data = encoder.transform(movies2)
    # We need the data as DataFrame + lowercase
    original_column_names = encoder.columns_
    column_names = list(map(lambda x:x.lower(), encoder.columns_))
    prepared_data = pd.DataFrame(prepared_data, columns=column_names)

    # create the candidates with apriori-algorithm
    candidates = apriori(prepared_data, min_support=0.02, use_colnames=True)

    # generate strong rules (Persons who watch X watch Y)
    rules = association_rules(candidates, metric='confidence', min_threshold=0.05, support_only=False)

    # get all proposals of the film/series
    proposals = set()
    for proposal in rules[rules['antecedents'] == {name}]['consequents']:
        for p in proposal:
            # get uppercase name:
            i = column_names.index(p)
            proposals.add(original_column_names[i])    # use the original -> not lower-case names
    return proposals

In [12]:
def proposal_console(name=None):
    # 2 Modi -> interactive or 1 Film/serie
    #     -> given argument declares which mode is used
    #                 argument = None     => interactive mode
    #                 argument = Filmname => single mode

    # Start interactive Console -> if no filmaname is given
    if name == None:
        # while not type exit or x
        while True:
            print("\n(Type exit or x for exit)\n")
            user_input = input("Film or Series:")
            if user_input == 'exit' or user_input == 'x':
                print('Thank you for using the Proposal Console. See you next time.')
                break
            # get products, which watched by people who watched the input product
            result = get_proposals(user_input)
            # is there a result for this film/series
            if len(result) > 0:
                result_txt = ''
                for entry in result:
                    result_txt += f"'{entry}', "
                result_txt = result_txt[:-2] + "."
                print(f"Persons who watched '{user_input}' also watched {result_txt}")
            else:
                print(f"There are no proposals for the film/series '{user_input}'.\nMake sure the Film/Series exist and is written right.")
                print("For exit the console write 'exit' or 'x'")
    # Start one Proposal -> no interactive console
    else:
        # get products, which watched by people who watched the input product
        result = get_proposals(name)
        # is there a result for this film/series
        if len(result) > 0:
            result_txt = ''
            for entry in result:
                result_txt += f"'{entry}', "
            result_txt = result_txt[:-2] + "."
            print(f"Persons who watched '{name}' also watched {result_txt}")
        else:
            print(f"There are no proposals for the film/series '{name}'.\nMake sure the Film/Series exist and is written right.")


In [13]:
proposal_console()


(Type exit or x for exit)

Persons who watched 'Mr. Robot' also watched 'Two and a half men', 'The Blacklist', 'Ozark', 'Atypical', 'Sex Education'.

(Type exit or x for exit)

There are no proposals for the film/series 'sdfdsf'.
 Make sure the Film/Series exist and is written right.
For exit the console write 'exit' or 'x'

(Type exit or x for exit)

Thank you for using the Proposal Console. See you next time.


In [14]:
proposal_console('Two and a half Men')

Persons who watched 'Two and a half Men' also watched 'Stranger Things', 'The Walking Dead', 'Daredevil', 'The Blacklist', 'Ozark', 'Atypical', 'Mr. Robot', 'Sex Education', 'Outer Banks'.


In [17]:
proposal_console('NoFilm')

There are no proposals for the film/series 'NoFilm'.
 Make sure the Film/Series exist and is written right.


#### Teilaufgabe c) Begründung zur Wahl der Parameter / Schwellwerte (2 Punkte)

Die Parameter minSamples und minConfidence wurden recht klein gewählt, wegen der Domaine.<br>
Viele Filme und Serien werden von Menschen angeschaut, welche nicht unbedingt Fan von ähnlichen Filmen/Serien sind. Dies könnte an der Popularität der Serie/Film oder auch an der besonders guten Qualität bzw. Originalität der Serie/Film liegen. Damit muss man die minSamples sowie die minConfi geringer Einstellen. 
<br>In anderen Worten: Es kommt also häufiger vor, dass Person X Film Y schaut, obwohl ihn keine Thriller interessieren und Person X sich danach keine weiteren Thriller Filme ansieht.<br>
<br>
Die Bewertungsart wurde auf dem standart 'Confidence' gelassen und dafür gibt es auch keinen sehr tiefgehenden Grund. Eigene empirische Versuche konnten gute Ergebnisse liefern und da es damit keine Mängel oder Lösungsbdarf gab, wurde dieser Parameter endgültig so eingestellt.

#### Teilaufgabe d) Wenden Sie nun Ihr System an, um eine Empfehlung zu generieren für Nutzer, die die Sendung "Mr. Robot" geschaut haben (1 Punkt)

In [15]:
proposal_console('Mr. Robot')

Persons who watched 'Mr. Robot' also watched 'Two and a half men', 'The Blacklist', 'Ozark', 'Atypical', 'Sex Education'.


In [16]:
proposal_console('mr. robot')

Persons who watched 'mr. robot' also watched 'Two and a half men', 'The Blacklist', 'Ozark', 'Atypical', 'Sex Education'.


#### Optional GUI-Version of Proposal:

In [7]:
import tkinter as tk
from tkinter import ttk

def search_button_event(name:str, output_var, root):
    result = get_proposals(name)
    if len(result) > 0:
        result_txt = ''
        for entry in result:
            result_txt += f"'{entry}', "
        result_txt = result_txt[:-2] + "."
        output = f"Persons who watched '{name}' also watched {result_txt}"
    else:
        output = f"There are no proposals for the film/series '{name}'.\nMake sure the Film/Series exist and is written right."
    output_var.set(f"Output:\n{output}")
    update_size(root)

def update_size(root):
    root.minsize(0, 0)
    width = root.winfo_width()
    height = root.winfo_height()
    root.geometry('')
    root.update()
    root.minsize(root.winfo_width(), root.winfo_height())
    root.geometry(f"{width}x{height}")

root = tk.Tk()
root.title("Watch Proposal")
root.geometry("600x400")
#root.minsize(400, 200)

main_window = ttk.Frame(root)
main_window.pack(expand=True, fill='both')

input_label = ttk.Label(main_window, text="User-Input:")
input_label.grid(row=1, column=1, sticky="nswe", pady=10, padx=20)

user_input = tk.StringVar()
input_entry = ttk.Entry(main_window, textvariable=user_input)
input_entry.grid(row=1, column=2, sticky="we", pady=10, padx=20)

output_var = tk.StringVar()
output_var.set("Output:")
output_label = ttk.Label(main_window, textvariable=output_var, borderwidth=2)
output_label.grid(row=3, rowspan=2, column=1, columnspan=2, sticky="nswe", pady=10, padx=20)

search_button = ttk.Button(main_window, text="search", command=lambda: search_button_event(user_input.get(), output_var, root), takefocus=0)
search_button.grid(row=2, column=1, columnspan=2, sticky="nswe", ipady=10, padx=20)

# set weights for resizable
for i in range(6):
    main_window.grid_rowconfigure(i, weight=1)
for i in range(4):
    main_window.grid_columnconfigure(i, weight=1)

update_size(root)
root.geometry("600x400")
root.mainloop()